In [1]:
import os
import openai
import panel as pn
from dotenv import load_dotenv, find_dotenv
from IPython.display import HTML
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role":"user","content":prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = 0,
    )
    return response.choices[0].message["content"]
def get_completion_from_messages(messages,model="gpt-3.5-turbo",temperature=0):
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature,
    )
    return response.choices[0].message["content"]

In [9]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({"role":"user","content":prompt})
    response = get_completion_from_messages(context)
    context.append({"role":"assistant","content":f"{response}"})
    panels.append(
        pn.Row("user",pn.pane.Markdown(prompt,width=600)))
    panels.append(
        pn.Row("assistant",pn.pane.Markdown(prompt,width=600,styles={'background-color':"grey"})))
    return pn.Column(*panels)

In [ ]:
pn.extension()
panels = []

context = [{"role":"system","content":"""
you are an resumebuilderbot, an automated service to collect information of user to build resume,\
you first greet the user then collect the information, \
be short for your answer and user friendly voice, \
ask for his or her name, mobile number, email address one by one, \
then ask for objective ask if user want to write its own or want is to be bot generative, \
if bot generative generate 50 words objective for user, \
then ask for education qualifications, \
which include section like qualification, passing year, School and percentage, ask for each section one by one,\
after entering percentage ask user wheather he or she want to add more qualification, \
if yes then same questions again. \
then ask for his key strengths, \
then ask for experiance if the user have any else move on to next section, \
if yes ask for position, company name, years/months one by one, \
if have any other experiance ask for yes or no, \
then same above questions again, \
after this ask for technical skills the user have, \
and then ask for some personal details, \
like gender, nationality, martial status, languages known, date of birth etc, \
After getting all this information ask user if he or she want to have any changes in any particular section.\
if yes ask for section and update the information that will be given by the user.\
After getting all information ask the user about the style of resume give some names to the user for style format.\
always remember until the user doesn't say you to move to further section stick to that particular section.
"""}]

inp = pn.widgets.TextInput(value = "hi",placeholder="Enter your text...")
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages,button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator = True,height=300),
)

dashboard

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a html code of resume design with previous details. create table for \
 education and experiance section, \
 main heading should be Resume then line by line add other info,\
 and do not include those empty information that are not provided by user, \
 section should be made in order like a official resume,\
 every thing should be well arranged and bulleted if needed,\
 and at end also include signature and date field,\
 also ensure that the design for the resume should be from the style choosen by user,\
 also create border outside the resume'},    
)
response = get_completion_from_messages(messages, temperature=0)
display(HTML(response))